# QuPDE usage examples

QuPDE is a Python library that finds an optimal and monomial quadratic transformation (quadratization) for nonquadratic PDEs using Sympy. QuPDE handles one-dimensional PDEs that are polynomial or rational. We present some usage examples where we showcast some functionalities of our algorithm.

First we import Sympy and QuPDE 

In [17]:
import sympy as sp
from qupde.quadratize import quadratize

## General usage
### Dym equation 

The Harry Dym equation describes the dynamics of several physical systems. This equation
is completely integrable and represents a system in which dispersion and nonlinearity are coupled
together: 

$$u_{t}=u^{3}u_{xxx}.$$

Let us find a quadratization for the Dym equation using QuPDE. First, we write the differential equation using Sympy objects:

In [2]:
t, x = sp.symbols('t x')
u = sp.Function('u')(t,x)

u_t = u**3 * sp.Derivative(u, x, 3)

To find a quadratization for a PDE we use the main function of the software called *quadratize*. This function receives as input the PDE as a list of tuples, where each tuple represents a differential equation. The first entry of each tuple is an undefined function and the second entry is its corresponding differential equation right-hand side. In our example: 

In [30]:
quadratize([(u, u_t)])

([u**3, u*u_x1**2], [], 21)

This function returns a tuple with:
- A list with the auxiliary polynomial variables
- A list with the introduced rational variables 
- The number of quadratizations attempted (nodes visited)

Now, if we want to see the quadratization and the transformed PDE in a more readable format, we call the same function but with the optional *printing* parameter with the available printing options: 
- `'pprint'` for pretty printing (Sympy's functionality).
- `'latex'` for printing the result in LaTeX format. 

In [31]:
quadratize([(u, u_t)], printing='pprint')


Quadratization:
      3
w₀ = u 
          2
w₁ = u⋅uₓ₁ 

Quadratic PDE:
                             2⋅w₀ₓ₁⋅w₀ₓ₂
w₀ₜ = w₀⋅w₀ₓ₃ - 10⋅w₀⋅w₁ₓ₁ + ───────────
                                  9     
                2⋅w₀ₓ₂⋅w₀ₓ₃                                       
w₁ₜ = w₀⋅w₁ₓ₃ - ─────────── + 4⋅w₀ₓ₂⋅w₁ₓ₁ + 4⋅w₀ₓ₃⋅w₁ - 24⋅w₁⋅w₁ₓ₁
                     3                                            
uₜ = uₓ₃⋅w₀


([u**3, u*u_x1**2], [], 21)

In [32]:
quadratize([(u, u_t)], printing='latex')


Quadratization:
w_{0} = u^{3}
w_{1} = u u_{x1}^{2}

Quadratic PDE:
w_{0t} = w_{0} w_{0x3} - 10 w_{0} w_{1x1} + \frac{2 w_{0x1} w_{0x2}}{9}
w_{1t} = w_{0} w_{1x3} - \frac{2 w_{0x2} w_{0x3}}{3} + 4 w_{0x2} w_{1x1} + 4 w_{0x3} w_{1} - 24 w_{1} w_{1x1}
u_{t} = u_{x3} w_{0}


([u**3, u*u_x1**2], [], 21)

## *Quadratize* parameters

The *quadratize* function has additional parameters that the user can modify. These correspond to: 
1. The number of spatial derivatives of the auxiliary variables to include in the quadratic transformations 
2. The heuristic to sort each set of new variables introduced when searching for an optimal quadratization.
3. The bound of the maximum number of variables to explore. 
4. The symbol of the first independent variable (default is *t*). 
5. The maximum order of spatial derivatives of the unknown function allowed within the auxiliary variables. 
6. The search algorithm.

We offer examples to show how some of them work. 

### 1. Change the number of spatial derivatives of the auxiliary variables
By default this value is set to 3. It is important to note that the number of spatial derivatives taken directly affects the algorithm's ability to find a quadratization. For example, If we set this value to 2 for the Dym equation, we obtain an unsuccesful search. 

In [33]:
quadratize([(u, u_t)], n_diff=2)

Quadratization not found


([], 109)

In some cases, increasing this parameter will be required to obtain a quadratization. 

### 1. Change sorting heuristic
In the algorithm, there are three heuristics implemented: 
- By order of derivatives and total degree of the monomials (`by_order_degree`)
- By total degree and order of derivatives of the monomials (`by_degree_order`). 
- By the function: $degree + 2 \cdot order$ (`by_fun`).

The default option implemented is `by_fun`. If we want to use the sorting function `by_order_degree` to find a quadratization, we run

In [4]:
quadratize([(u, u_t)], sort_fun='by_order_degree', max_der_order=3, printing='pprint')


Quadratization:
      3
w₀ = u 
          2
w₁ = u⋅uₓ₁ 

Quadratic PDE:
w₀ₜ = w₀⋅w₀ₓ₃ - 2⋅w₀ₓ₁⋅w₀ₓ₂ + 10⋅w₀ₓ₁⋅w₁
                              2⋅w₀ₓ₂⋅w₀ₓ₃                           
w₁ₜ = w₀⋅w₁ₓ₃ + 2⋅w₀ₓ₁⋅w₁ₓ₂ - ─────────── + 2⋅w₀ₓ₂⋅w₁ₓ₁ + 12⋅w₁⋅w₁ₓ₁
                                   3                                
uₜ = uₓ₃⋅w₀


([u**3, u*u_x1**2], [], 21)

### 2. Change the bound of maximum number of new variables

The default number for this parameter is 10. If we want to quadratize a PDE that is simple in terms of polynomial degrees, we may want to decrease the bound on the number of variables to find the optimal quadratization faster. Or if a PDE has higher degrees, we may want to increase this bound. To do this, we just arbitrarly set the parameter `nvars_bound`. In our example, if we change this bound to 4, we obtain an optimal quadratization faster:

In [6]:
quadratize([(u, u_t)], nvars_bound=4, printing='pprint')


Quadratization:
      3
w₀ = u 
          2
w₁ = u⋅uₓ₁ 

Quadratic PDE:
w₀ₜ = w₀⋅w₀ₓ₃ - 2⋅w₀ₓ₁⋅w₀ₓ₂ + 10⋅w₀ₓ₁⋅w₁
                              2⋅w₀ₓ₂⋅w₀ₓ₃                           
w₁ₜ = w₀⋅w₁ₓ₃ + 2⋅w₀ₓ₁⋅w₁ₓ₂ - ─────────── + 2⋅w₀ₓ₂⋅w₁ₓ₁ + 12⋅w₁⋅w₁ₓ₁
                                   3                                
uₜ = uₓ₃⋅w₀


([u**3, u*u_x1**2], [], 21)

### 3. Change the symbol for the first independent variable

By default, this parameter is *t*. If we want to change the symbol of the first independent variable, we set the parameter `first_indep` equal to the new symbol. Note that changing the symbol for the second independent variable can be done just by defining the equation in Sympy with this new symbol. 

In [26]:
z=sp.symbols('z')
y=sp.symbols('y')
v=sp.Function('v')(z,y)

vz = v**3 * sp.Derivative(v, y, 3)

quadratize([(v, vz)], first_indep=z, printing='pprint')


Quadratization:
      3
w₀ = v 
           2
w₁ = v⋅v_y1 

Quadratic PDE:
                                2⋅w_0y1⋅w_0y2
w_0z = w₀⋅w_0y3 - 10⋅w₀⋅w_1y1 + ─────────────
                                      9      
                  2⋅w_0y2⋅w_0y3                                           
w_1z = w₀⋅w_1y3 - ───────────── + 4⋅w_0y2⋅w_1y1 + 4⋅w_0y3⋅w₁ - 24⋅w₁⋅w_1y1
                        3                                                 
v_z = v_y3⋅w₀


([v**3, v*v_y1**2], [], 21)

### 4. Change the maximum order of derivatives

By default, we prune every branch that introduces spatial derivatives of higher order than those within the original equations. To change this parameter, we set `max_der_order` to the desired maximum order. An important note is that in some cases, we need to relax this limit to obtain a quadratization for a PDE. For example, if run *quadratize* for the equation

In [22]:
ut2 =  sp.Derivative(u, x)**3 + u**3
quadratize([(u, ut2)], printing = 'pprint')

Quadratization not found


([], 9)

the algorithm does not find a quadratization. Now, if we relax the maximum order of derivatives rule and allow derivatives up to order 3, we obtain a different result 

In [23]:
quadratize([(u, ut2)], max_der_order=3, printing = 'pprint')


Quadratization:
      2
w₀ = u 
        2
w₁ = uₓ₁ 

Quadratic PDE:
          2          
w₀ₜ = 2⋅w₀  + w₀ₓ₁⋅w₁
w₁ₜ = 6⋅w₀⋅w₁ + 3⋅w₁⋅w₁ₓ₁
            u⋅w₀ₓ₃   uₓ₁⋅w₀ₓ₂   uₓ₃⋅w₀
uₜ = u⋅w₀ - ────── + ──────── + ──────
              6         2         3   


([u**2, u_x1**2], [], 7)

### 5. Change the search algorithm 

The default algorithm for searching a quadratization is the branch-and-bound framework. To change this, set the parameter `search_alg` to either `'bnb'` for branch-and-bound, or `'inn'` for the incremental nearest neighbor implementation. 

In [28]:
quadratize([(v, vz)], first_indep=z, search_alg='inn', printing='pprint')


Quadratization:
      3
w₀ = v 
           2
w₁ = v⋅v_y1 

Quadratic PDE:
                                2⋅w_0y1⋅w_0y2
w_0z = w₀⋅w_0y3 - 10⋅w₀⋅w_1y1 + ─────────────
                                      9      
                  2⋅w_0y2⋅w_0y3                                           
w_1z = w₀⋅w_1y3 - ───────────── + 4⋅w_0y2⋅w_1y1 + 4⋅w_0y3⋅w₁ - 24⋅w₁⋅w_1y1
                        3                                                 
v_z = v_y3⋅w₀


([v**3, v*v_y1**2], [], 10)

## Other examples
### Allen-Cahn equation

First, we run QuPDE for the Allen-Cahn equation, described by the PDE $$u_t = u_{xx} + u - u^3.$$ This time, we print the result in LaTeX format. 

In [ ]:
t, x = sp.symbols('t x')
u = sp.Function('u')(t,x)

u_t = D(u, x, 2) + u - u**3 

quadratize([(u, u_t)], printing='latex')

### FitzHugh-Nagamo system

The FitzHugh-Nagamo system is a simplified neuron model of the Hodgkin-Huxley model, which describes activation and deactivation dynamics of a spiking neuron. Its governing equations are

$$ v_t = \epsilon v_{xx} + \dfrac{1}{\epsilon}v(v - 0.1)(1 - v) - \dfrac{1}{\epsilon}u + \dfrac{1}{\epsilon}q, $$
$$ u_t = hv - \gamma u + q. $$

We define the symbolic coefficients first to use QuPDE on this equation. 

In [29]:
t, x = sp.symbols('t x')
v = sp.Function('v')(t,x)
y = sp.Function('y')(t,x)
epsilon, h, gamma, r = sp.symbols('epsilon h gamma r', constant=True)

v_t = epsilon * sp.Derivative(v, x, 2) - (1/epsilon) * (v * (v - 0.1) * (1 - v)) - y/epsilon + r/epsilon
y_t = h * v - gamma * y + r

quadratize([(v, v_t), (y, y_t)], search_alg='bnb', printing='pprint')


Quadratization:
      2
w₀ = v 

Quadratic PDE:
         2                 11⋅v⋅w₀               2   w₀
w₀ₜ = 2⋅ε ⋅v⋅vₓ₂ + 2⋅r⋅v - ─────── - 2⋅v⋅y + 2⋅w₀  + ──
                              5                      5 
      2                  v    11⋅w₀    
vₜ = ε ⋅vₓ₂ + r + v⋅w₀ + ── - ───── - y
                         10     10     
yₜ = -γ⋅y + h⋅v + r


([v**2], [], 3)